In [1]:
import pandas as pd
import numpy as np
import math
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import requests

# Instructions for installing MATALB engine: https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html
import matlab.engine

ModuleNotFoundError: No module named 'matlab'

In [2]:
# Reading in Data
csrData = pd.read_excel('csrData.xlsx',skiprows=[0,1,3])
dsrData = pd.read_excel('dsrData.xlsx',skiprows=[0,1,3])
leverageData = pd.read_excel('leverageData.xlsx',skiprows=[0,1,3])
waccData = pd.read_excel('waccData.xlsx',skiprows=[0,1,3])
stock_universeData = pd.read_excel('stockuniverseData.xlsx',skiprows=[0,1])
# AAA yield data from: https://fred.stlouisfed.org/series/AAA
aaa_yieldData = pd.read_csv('AAA.csv',index_col=0)
# BAA yield data from: https://fred.stlouisfed.org/series/BAA
baa_yieldData = pd.read_csv('BAA.csv',index_col=0)
yieldData = pd.concat([aaa_yieldData,baa_yieldData], axis=1)
yieldData['BAA_2'] = yieldData['BAA'] + 2
chsData = pd.read_csv('chsData.csv')
ticker_to_permno = pd.read_excel('ticker_to_permno.xlsx')

# Fill in missing values for COGS and SGA with 0
stock_universeData = stock_universeData[~(stock_universeData['GICS Sector'] == 'Real Estate')]
stock_universeData = stock_universeData[~(stock_universeData['GICS Sector'] == 'Utilities')]
stock_universeData = stock_universeData[stock_universeData['Ticker'].str.contains('US Equity', regex=False)].reset_index(drop=True)
stock_universeData = stock_universeData.sort_values(by='Market Cap',ascending=False).reset_index(drop=True)
stock_universeData.fillna({'SG&A Expn T12M':0,'COGS/FE&PP&G T12M':0}, inplace=True)

In [3]:
# Calculating average industry CSR over past 3 years by SIC Code
csrData = csrData[csrData['SIC Code'].notnull()].reset_index(drop=True)
csrData['SIC_Code'] = csrData['SIC Code'].astype(str).str[:2]
csrData = csrData[(csrData['Revenue T12M'] != 0) & (csrData['Revenue:CQ-4 T12M'] != 0) & (csrData['Revenue:CQ-8 T12M'] != 0)]
csrData['1yr_back_csr'] = csrData['CAPEX T12M'] / csrData['Revenue T12M']
csrData['2yr_back_csr'] = csrData['CAPEX:CQ-4 T12M'] / csrData['Revenue:CQ-4 T12M']
csrData['3yr_back_csr'] = csrData['CAPEX:CQ-8 T12M'] / csrData['Revenue:CQ-8 T12M']
csrData['CSR'] = csrData[['1yr_back_csr','2yr_back_csr','3yr_back_csr']].mean(axis=1) * -1
csr = csrData[['SIC_Code','CSR']].groupby(['SIC_Code'], as_index=False).mean()

In [4]:
# Calculating average industry DSR over past 3 years by SIC Code
dsrData = dsrData[dsrData['SIC Code'].notnull()].reset_index(drop=True)
dsrData['SIC_Code'] = dsrData['SIC Code'].astype(str).str[:2]
dsrData = dsrData[(dsrData['Revenue T12M'] != 0) & (dsrData['Revenue:CQ-4 T12M'] != 0) & (dsrData['Revenue:CQ-8 T12M'] != 0)]
dsrData['1yr_back_dsr'] = dsrData['D&A T12M'] / dsrData['Revenue T12M']
dsrData['2yr_back_dsr'] = dsrData['D&A:CQ-4 T12M'] / dsrData['Revenue:CQ-4 T12M']
dsrData['3yr_back_dsr'] = dsrData['D&A:CQ-8 T12M'] / dsrData['Revenue:CQ-8 T12M']
dsrData['DSR'] = dsrData[['1yr_back_dsr','2yr_back_dsr','3yr_back_dsr']].mean(axis=1)
dsr = dsrData[['SIC_Code','DSR']].groupby(['SIC_Code'],as_index=False).mean()

In [5]:
# Calculating average industry leverage ratio over past 3 years by SIC Code
leverageData = leverageData[leverageData['SIC Code'].notnull()].reset_index(drop=True)
leverageData['SIC_Code'] = leverageData['SIC Code'].astype(str).str[:2]
leverageData['1yr_back_levratio'] = (leverageData['ST Brrwng:Q'] + leverageData['LT Brrwng:Q']) / (leverageData['ST Brrwng:Q'] + leverageData['LT Brrwng:Q'] + leverageData['Market Cap:Q'])
leverageData['2yr_back_levratio'] = (leverageData['ST Brrwng:Q-4'] + leverageData['LT Brrwng:Q-4']) / (leverageData['ST Brrwng:Q-4'] + leverageData['LT Brrwng:Q-4'] + leverageData['Market Cap:Q-4'])
leverageData['3yr_back_levratio'] = (leverageData['ST Brrwng:Q-8'] + leverageData['LT Brrwng:Q-8']) / (leverageData['ST Brrwng:Q-8'] + leverageData['LT Brrwng:Q-8'] + leverageData['Market Cap:Q-8'])
leverageData['LevRatio'] = leverageData[['1yr_back_levratio','2yr_back_levratio','3yr_back_levratio']].mean(axis=1)
levratio = leverageData[['SIC_Code','LevRatio']].groupby(['SIC_Code'],as_index=False).mean()

In [6]:
# Viewing average industry CSR, DSR, and leverage ratio by SIC Code
csrDsrLevratioBySIC = csr.merge(dsr,on='SIC_Code').merge(levratio,on='SIC_Code')

In [7]:
# Annualizing quarterly volatility of sales using coefficient of variation and calculating gross margin
stock_universeData['Sigma'] = stock_universeData[['Revenue:Q-1','Revenue:Q-2','Revenue:Q-3','Revenue:Q-4','Revenue:Q-5','Revenue:Q-6','Revenue:Q-7','Revenue:Q-8']].std(axis=1) * math.sqrt(4)
stock_universeData['Mean'] = stock_universeData[['Revenue:Q-1','Revenue:Q-2','Revenue:Q-3','Revenue:Q-4','Revenue:Q-5','Revenue:Q-6','Revenue:Q-7','Revenue:Q-8']].mean(axis=1)
stock_universeData['Volatility'] = stock_universeData.Sigma / stock_universeData.Mean
stock_universeData['Gross Margin'] = stock_universeData['Revenue T12M'] - stock_universeData['COGS/FE&PP&G T12M']                     

In [8]:
# Getting rid of some weirdness and connecting PERMNO to Ticker
stock_universeData['SIC_Code'] = stock_universeData['SIC Code'].astype(str).str[:2]
stock_universeData['TICKER'] = stock_universeData['Ticker'].str.replace(r' [A-Z][A-Z] Equity', '')
stock_universeData['TICKER'] = stock_universeData['TICKER'].str.replace(r'/[A-Z]', '', regex=True)
keeping_duplicate_with_highest_volume = stock_universeData.groupby('TICKER')['Average Volume:D-30'].transform(max) == stock_universeData['Average Volume:D-30']
stock_universeData2 = stock_universeData[keeping_duplicate_with_highest_volume]
latest_permno_for_ticker = ticker_to_permno.groupby(['TICKER'])['DATE'].transform(max) == ticker_to_permno['DATE']
ticker_to_permno2 = ticker_to_permno[latest_permno_for_ticker]
stock_universeData3 = pd.merge(stock_universeData2, ticker_to_permno2, on='TICKER', how='left')

In [9]:
# Getting latest CHS Quintile for a PERMNO and connecting with Ticker
chsData2 = chsData[['PERMNO','DATE','CHS_quintile']]
latest_CHS_for_permno = chsData2.groupby(['PERMNO'])['DATE'].transform(max) == chsData2['DATE']
chsData3 = chsData2[latest_CHS_for_permno]
stock_universeData4 = pd.merge(stock_universeData3,chsData3,on='PERMNO', how='left')

In [10]:
# Calculating coupon rate
# Treasury yield info from: https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield
treasuryYield_3mo = 1.55 / 100
treasuryYield_10yr = 1.51 / 100
curr_month = '2020-01-01'
rf = (treasuryYield_3mo + treasuryYield_10yr) / 2

stock_universeData4['rf'] = rf
stock_universeData4['coupon_rate'] = np.nan
stock_universeData4.loc[stock_universeData4['CHS_quintile'] == 1, 'coupon_rate'] = ((yieldData.loc[curr_month,'AAA'] / 100) + rf)
stock_universeData4.loc[stock_universeData4['CHS_quintile'] == 2, 'coupon_rate'] = ((yieldData.loc[curr_month,'AAA'] / 100) + rf)
stock_universeData4.loc[stock_universeData4['CHS_quintile'] == 3, 'coupon_rate'] = ((yieldData.loc[curr_month,'BAA'] / 100)+ rf)
stock_universeData4.loc[stock_universeData4['CHS_quintile'] == 4, 'coupon_rate'] = ((yieldData.loc[curr_month,'BAA'] / 100) + rf)
stock_universeData4.loc[stock_universeData4['CHS_quintile'] == 5, 'coupon_rate'] = ((yieldData.loc[curr_month,'BAA_2'] / 100) + rf)

In [11]:
# Connecting WACC Data with CHS Quintiles
waccData['SIC_Code'] = waccData['SIC Code'].astype(str).str[:2]
waccData['TICKER'] = waccData['Ticker'].str.replace(r' [A-Z][A-Z] Equity', '')
waccData['TICKER'] = waccData['TICKER'].str.replace(r'/[A-Z]', '', regex=True)
keeping_duplicate_with_highest_price = waccData.groupby('TICKER')['Market Cap'].transform(max) == waccData['Market Cap']
waccData2 = waccData[keeping_duplicate_with_highest_price]
waccData3 = pd.merge(waccData2,ticker_to_permno2,on='TICKER', how='left')
waccData4 = pd.merge(waccData3,chsData3,on='PERMNO', how='left')

In [12]:
# Calculating average industry distress betas
average_industry_distress_betas = waccData4.groupby(['SIC_Code','CHS_quintile'],as_index=False)['Raw Beta:Y-3'].mean()
stock_universeData5 = pd.merge(stock_universeData4, average_industry_distress_betas, on=['SIC_Code','CHS_quintile'], how='left')

In [13]:
# Calculating cost of equity using CAPM
market_risk_premium = 0.05
stock_universeData5['cost_of_equity'] = rf + (stock_universeData5['Raw Beta:Y-3'] * market_risk_premium)

In [14]:
# Calculating Ra,return on assets as described on pg. 855
tax_rate = 0.21

stock_universeData5['Total Debt'] = stock_universeData5['ST Brrwng LF'] + stock_universeData5['LT Brrwng LF']
stock_universeData5['Total Value'] = stock_universeData5['ST Brrwng LF'] + stock_universeData5['LT Brrwng LF'] + stock_universeData5['Market Cap']
stock_universeData5['Ra']=((stock_universeData5['Market Cap']/stock_universeData5['Total Value'])*stock_universeData5['cost_of_equity'])+((stock_universeData5['Total Debt']/stock_universeData5['Total Value'])*stock_universeData5['coupon_rate']*tax_rate)

In [15]:
# Saving information into a modelInputs dataframe/file and doing some clean up
stock_universeData6 = pd.merge(stock_universeData5,csrDsrLevratioBySIC,on='SIC_Code',how='left')
modelInputs = stock_universeData6[['TICKER','Market Cap','Price:D-1','GICS Sector', 'SIC_Code','CHS_quintile','PERMNO','ST Brrwng LF','LT Brrwng LF','Revenue T12M','SG&A Expn T12M','Volatility','rf','coupon_rate','Gross Margin','Ra','CSR','DSR','LevRatio']]
modelInputs_dropping_missing_information = modelInputs.dropna().reset_index(drop=True)
modelInputs_dropping_missing_information2 = modelInputs_dropping_missing_information
modelInputs_dropping_missing_information2[['ST Brrwng LF','LT Brrwng LF','Revenue T12M','SG&A Expn T12M','Gross Margin']] = modelInputs_dropping_missing_information[['ST Brrwng LF','LT Brrwng LF','Revenue T12M','SG&A Expn T12M','Gross Margin']].div(1000000)
modelInputs_dropping_missing_information2['Valuations'] = np.nan

modelInputs_dropping_missing_information3 = modelInputs_dropping_missing_information2[modelInputs_dropping_missing_information2.groupby('TICKER')['CHS_quintile'].transform(max) == modelInputs_dropping_missing_information2['CHS_quintile']]
modelInputs_dropping_missing_information4 = modelInputs_dropping_missing_information3[modelInputs_dropping_missing_information3.groupby('TICKER')['PERMNO'].transform(max) == modelInputs_dropping_missing_information3['PERMNO']]

In [16]:
modelInputs_dropping_missing_information4.to_csv('modelInputs.csv',index=False)

In [17]:
# Running model and saving valuations into new dataframe
eng = matlab.engine.start_matlab()

In [ ]:
# Make sure to enable scrolling for the output for this cell
for index, row in modelInputs_dropping_missing_information4.iterrows():
    valuation = eng.model_matlab(row['ST Brrwng LF'],row['LT Brrwng LF'],row['Revenue T12M'],row['SG&A Expn T12M'],row['Volatility'],row['rf'],row['coupon_rate'],row['Gross Margin'],row['Ra'],row['CSR'],row['DSR'],row['LevRatio'])
    valuation = valuation * 1000000 #Bringing it back to same units as Market Cap
    print(valuation, index) # uncomment this line if you would like to see progress of the valuations, should take ~3min or less
    modelInputs_dropping_missing_information4.at[index,'Valuations'] = valuation

-3333000000.0 0
-3333000000.0 1
-3333000000.0 2
-3333000000.0 3
-3333000000.0 4
-3333000000.0 5
-3333000000.0 6
-3333000000.0 7
-3333000000.0 8
-3333000000.0 9
74646743624.83507 10
-3333000000.0 11
-3333000000.0 12
-3333000000.0 13
-3333000000.0 14
-3333000000.0 15
-3333000000.0 16
-3333000000.0 17
-3333000000.0 18
-3333000000.0 19
-3333000000.0 20
-3333000000.0 21
-3333000000.0 22
81264262440.83398 23
-3333000000.0 24
-3333000000.0 25
-3333000000.0 26
-3333000000.0 27
78332846147.64517 28
43510324941.40881 29
-3333000000.0 30
35825555286.57159 31
95606843990.13707 32
-3333000000.0 33
35741100273.46509 34
42555800430.81955 35
119922431732.01259 36
164766848556.92377 37
37796505176.02324 38
50272312512.746376 39
113683017449.74815 40
-3333000000.0 41
-3333000000.0 42
150266127187.6598 43
275132028555.8853 44
-3333000000.0 45
166504366216.65936 46
25520372589.827915 47
-3333000000.0 48
57075027962.70495 49
-3333000000.0 50
81115196652.14067 51
138168018195.54373 52
51940844825.24038 53
1

In [ ]:
# Computing deciles and outputting excel file
modelOutputs = modelInputs_dropping_missing_information4
modelOutputs['Valuation/MarketCap'] = modelOutputs['Valuations'] / modelOutputs['Market Cap']
modelOutputs2 = modelOutputs[modelOutputs.Valuations > 0].reset_index(drop=True)
modelOutputs3 = modelOutputs2.sort_values(by=['Valuation/MarketCap'],ascending=False)
modelOutputs4 = modelOutputs3.iloc[(round(len(modelOutputs3)*(1/100))):(len(modelOutputs3)-round(len(modelOutputs3)*(1/100)))].reset_index(drop=True)
modelOutputs4['Decile'] = pd.qcut(modelOutputs4['Valuation/MarketCap'],10,labels=False) + 1

In [2]:
modelOutputs4 = pd.read_csv('modelOutputs.csv')

In [3]:
# Current sector weights for top 50 stocks
df = modelOutputs4[0:50]
df['GICS Sector'].value_counts(normalize=True) * 100

Financials                42.0
Consumer Discretionary    18.0
Communication Services    12.0
Industrials                8.0
Information Technology     6.0
Materials                  6.0
Health Care                6.0
Energy                     2.0
Name: GICS Sector, dtype: float64

In [4]:
# Constructing portfolio and keeping industry weights within charter criteria
i = 50
percentage = 20.0

count = 0
for each in df['GICS Sector'].unique():
    count += 1
countCheck = 0
df = modelOutputs4[0:i]

while countCheck < 9:
    # Checks for overweight sectors, drops the last row for sector, and adds a row from modelOutputs4
    print(countCheck,count)
    countCheck = 0
    count = 0
    for each in df['GICS Sector'].unique():
        count += 1
    for each in df['GICS Sector'].unique():
        j = round(((len(df[df['GICS Sector']==each])/len(df))*100),2)
        print(each,j)
        if j >= percentage:
            print('bad')
            asdf = df[df['GICS Sector']==each]
            df = df[~df.isin(asdf.tail(1))].dropna()
            i += 1
            df = df.append(modelOutputs4.iloc[i],ignore_index=True)
        else:
            print('ok')
            countCheck +=1

0 8
Materials 6.0
ok
Financials 42.0
bad
Health Care 6.0
ok
Consumer Discretionary 18.0
ok
Communication Services 12.0
ok
Industrials 8.0
ok
Information Technology 6.0
ok
Energy 2.0
ok
7 8
Materials 6.0
ok
Financials 42.0
bad
Health Care 6.0
ok
Consumer Discretionary 18.0
ok
Communication Services 12.0
ok
Industrials 8.0
ok
Information Technology 6.0
ok
Energy 2.0
ok
7 8
Materials 6.0
ok
Financials 42.0
bad
Health Care 6.0
ok
Consumer Discretionary 18.0
ok
Communication Services 12.0
ok
Industrials 8.0
ok
Information Technology 6.0
ok
Energy 2.0
ok
7 8
Materials 6.0
ok
Financials 42.0
bad
Health Care 6.0
ok
Consumer Discretionary 20.0
bad
Communication Services 12.0
ok
Industrials 8.0
ok
Information Technology 6.0
ok
Energy 2.0
ok
6 8
Materials 6.0
ok
Financials 40.0
bad
Health Care 6.0
ok
Consumer Discretionary 22.0
bad
Communication Services 12.0
ok
Industrials 8.0
ok
Information Technology 6.0
ok
Energy 2.0
ok
6 8
Materials 6.0
ok
Financials 38.0
bad
Health Care 8.0
ok
Consumer Disc

In [5]:
# Sector weights after filtering
df['GICS Sector'].value_counts(normalize=True) * 100

Consumer Discretionary    18.0
Financials                18.0
Industrials               16.0
Communication Services    14.0
Materials                 14.0
Health Care                8.0
Information Technology     6.0
Energy                     4.0
Consumer Staples           2.0
Name: GICS Sector, dtype: float64

In [6]:
df

,TICKER,Market Cap,Price:D-1,GICS Sector,SIC_Code,CHS_quintile,PERMNO,ST Brrwng LF,LT Brrwng LF,Revenue T12M,...,rf,coupon_rate,Gross Margin,Ra,CSR,DSR,LevRatio,Valuations,Valuation/MarketCap,Decile
0,TSE,1.118191e+09,29.559999,Materials,28.0,4.0,14755.0,27.600000,1220.300032,3952.000000,...,0.0153,0.0530,328.100096,0.039232,1.943539,1.056472,0.112478,3.524133e+10,31.516361,10.0
1,CNO,2.714473e+09,18.209999,Financials,63.0,2.0,89841.0,2797.799936,1056.600000,3725.000000,...,0.0153,0.0447,3725.000000,0.027410,0.010939,0.051491,0.203049,8.514556e+10,31.367256,10.0
2,MD,1.977641e+09,24.360001,Health Care,80.0,1.0,82272.0,23.611000,2004.464000,3540.862976,...,0.0153,0.0447,3540.862976,0.033362,0.044256,0.061591,0.289200,6.101144e+10,30.850608,10.0
3,SIG,1.263124e+09,24.180000,Consumer Discretionary,59.0,2.0,12076.0,329.900000,2237.700096,6138.499968,...,0.0153,0.0447,2203.600000,0.028505,0.034989,0.028247,0.275930,3.854309e+10,30.514090,10.0
4,LNC,1.079315e+10,55.680000,Financials,63.0,3.0,49015.0,311.000000,6270.000128,17444.000000,...,0.0153,0.0530,17444.000000,0.038950,0.010939,0.051491,0.203049,3.206460e+11,29.708301,10.0
5,AMCX,2.031208e+09,37.424999,Communication Services,48.0,3.0,12880.0,81.224000,3271.942912,3047.963008,...,0.0153,0.0530,3047.963008,0.028084,0.091598,0.132641,0.433814,5.958914e+10,29.336798,10.0
6,SC,9.058385e+09,26.590000,Financials,61.0,3.0,14414.0,10717.262850,39194.140670,7905.223936,...,0.0153,0.0530,7905.223936,0.016399,0.032929,0.043235,0.496239,2.646860e+11,29.219990,10.0
7,MDP,1.347823e+09,29.670000,Communication Services,27.0,1.0,42796.0,38.100000,2893.199872,3107.700032,...,0.0153,0.0447,1967.000032,0.025355,0.024297,0.058705,0.318204,3.878766e+10,28.777999,10.0
8,BIG,1.098273e+09,27.660000,Consumer Discretionary,53.0,2.0,67467.0,209.160992,1574.279040,5314.748928,...,0.0153,0.0447,2140.006976,0.025955,0.035126,0.033408,0.275663,3.155989e+10,28.735928,10.0
9,NGHC,2.505675e+09,22.120001,Financials,63.0,5.0,14464.0,696.060032,108.968000,4997.254016,...,0.0153,0.0730,4997.254016,0.049888,0.010939,0.051491,0.203049,7.043901e+10,28.111789,10.0


In [7]:
df.to_csv('modelOuputs_0217.csv',index=False)

In [8]:
portfolio = df[['TICKER','Market Cap','Price:D-1','GICS Sector','Valuation/MarketCap','Decile']]
# Saving portfolio as csv
# portfolio.to_csv('teamA_portfolio_Feb4.csv',index=False)

In [9]:
# Rows in outputs but not in portfolio
modelOutputs_minus_portfolio = modelOutputs4[~modelOutputs4.isin(df)].dropna().reset_index(drop=True)
modelOutputs_minus_portfolio.to_csv('modelOutputs_minus_portfolio.csv',index=False)

In [10]:
# Comparing previous holdings, seeing what is similar
prv_holdings = pd.read_excel('SIM Holdings.xlsx')
portfolio[portfolio['TICKER'].isin(prv_holdings['Ticker']).dropna()].reset_index(drop=True)

,TICKER,Market Cap,Price:D-1,GICS Sector,Valuation/MarketCap,Decile
0,PLAY,1.360714e+09,45.130001,Consumer Discretionary,16.310303,10.0
1,PBF,3.092082e+09,25.469999,Energy,12.866307,10.0
2,XPO,8.324559e+09,92.839996,Industrials,11.621469,10.0
3,OMC,1.646925e+10,76.730003,Communication Services,10.479927,10.0
4,ALGT,2.699647e+09,171.009995,Industrials,9.346030,10.0


In [11]:
prv_holdings

,Ticker,Shares,Price,Cost,Value,Fee
0,Y,18,796.1452,14339.56,14330.6136,8.9464
1,ALGT,84,171.4900,14414.11,14405.1600,8.9500
2,AFG,140,110.2895,15449.48,15440.5300,8.9500
3,UHAL,39,361.1200,14092.63,14083.6800,8.9500
4,AMKR,1300,12.6000,16399.50,16380.0000,19.5000
5,AXE,159,89.2200,14194.93,14185.9800,8.9500
6,ARNA,305,48.2800,14734.35,14725.4000,8.9500
7,CAR,476,31.1100,14817.31,14808.3600,8.9500
8,BECN,490,29.2100,14321.85,14312.9000,8.9500
9,BHE,500,35.2119,17614.90,17605.9500,8.9500


In [12]:
# Constructing sell orders
sell_orders = prv_holdings[~prv_holdings['Ticker'].isin(portfolio['TICKER']).dropna()].reset_index(drop=True)
sell_orders['Account ID'] = 1777463
sell_orders['Asset Identifier'] = sell_orders['Ticker']
sell_orders['Asset Identifier Type'] = 'TICKER'
sell_orders['Transaction'] = 'Sell'
sell_orders['Trade Date'] = '2/5/2020'
sell_orders['Amount'] = sell_orders['Shares']
sell_orders['Amount Type'] = 'SHARES'
sell_orders['Initial Purchase'] = 'N'
sell_orders2 = sell_orders[['Account ID','Asset Identifier','Asset Identifier Type','Transaction','Trade Date','Amount','Amount Type','Initial Purchase']]
sell_orders2.to_csv('trades_sellOrder_Portion.csv',index=False)
sell_orders2

,Account ID,Asset Identifier,Asset Identifier Type,Transaction,Trade Date,Amount,Amount Type,Initial Purchase
0,1777463,Y,TICKER,Sell,2/5/2020,18,SHARES,N
1,1777463,AFG,TICKER,Sell,2/5/2020,140,SHARES,N
2,1777463,UHAL,TICKER,Sell,2/5/2020,39,SHARES,N
3,1777463,AMKR,TICKER,Sell,2/5/2020,1300,SHARES,N
4,1777463,AXE,TICKER,Sell,2/5/2020,159,SHARES,N
5,1777463,ARNA,TICKER,Sell,2/5/2020,305,SHARES,N
6,1777463,CAR,TICKER,Sell,2/5/2020,476,SHARES,N
7,1777463,BECN,TICKER,Sell,2/5/2020,490,SHARES,N
8,1777463,BHE,TICKER,Sell,2/5/2020,500,SHARES,N
9,1777463,CWH,TICKER,Sell,2/5/2020,1072,SHARES,N


In [13]:
buy_orders = portfolio[~portfolio['TICKER'].isin(prv_holdings['Ticker']).dropna()].reset_index(drop=True)
buy_orders['Price'] = np.nan
buy_orders

,TICKER,Market Cap,Price:D-1,GICS Sector,Valuation/MarketCap,Decile,Price
0,TSE,1.118191e+09,29.559999,Materials,31.516361,10.0,NaN
1,CNO,2.714473e+09,18.209999,Financials,31.367256,10.0,NaN
2,MD,1.977641e+09,24.360001,Health Care,30.850608,10.0,NaN
3,SIG,1.263124e+09,24.180000,Consumer Discretionary,30.514090,10.0,NaN
4,LNC,1.079315e+10,55.680000,Financials,29.708301,10.0,NaN
5,AMCX,2.031208e+09,37.424999,Communication Services,29.336798,10.0,NaN
6,SC,9.058385e+09,26.590000,Financials,29.219990,10.0,NaN
7,MDP,1.347823e+09,29.670000,Communication Services,28.777999,10.0,NaN
8,BIG,1.098273e+09,27.660000,Consumer Discretionary,28.735928,10.0,NaN
9,NGHC,2.505675e+09,22.120001,Financials,28.111789,10.0,NaN


In [14]:
# Scraping prices from yahoo finance and updating Price column
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
for index, row in buy_orders.iterrows():
    ticker = row['TICKER']
    driver.get('https://finance.yahoo.com/quote/'+ticker)
    html = driver.page_source
    soup = BeautifulSoup(html,features="html.parser")
    x = soup.find('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})
    price = float(x.text)
    buy_orders.at[index,'Price'] = price
    print(index,price)
driver.close()

0 23.11
1 17.3
2 17.29
3 22.74
4 51.18
5 30.45
6 25.97
7 27.58
8 22.79
9 21.05
10 10.6
11 38.45
12 16.58
13 37.0
14 32.92
15 10.81
16 11.51
17 47.81
18 17.12
19 39.35
20 90.94
21 14.7
22 56.17
23 34.81
24 12.75
25 31.47
26 12.07
27 22.03
28 22.99
29 57.37
30 49.4
31 13.79
32 58.78
33 29.66
34 23.36
35 40.02
36 135.21
37 27.84
38 30.81
39 33.02
40 74.39
41 38.94
42 36.53
43 47.26
44 49.06


In [15]:
buy_orders

,TICKER,Market Cap,Price:D-1,GICS Sector,Valuation/MarketCap,Decile,Price
0,TSE,1.118191e+09,29.559999,Materials,31.516361,10.0,23.11
1,CNO,2.714473e+09,18.209999,Financials,31.367256,10.0,17.30
2,MD,1.977641e+09,24.360001,Health Care,30.850608,10.0,17.29
3,SIG,1.263124e+09,24.180000,Consumer Discretionary,30.514090,10.0,22.74
4,LNC,1.079315e+10,55.680000,Financials,29.708301,10.0,51.18
5,AMCX,2.031208e+09,37.424999,Communication Services,29.336798,10.0,30.45
6,SC,9.058385e+09,26.590000,Financials,29.219990,10.0,25.97
7,MDP,1.347823e+09,29.670000,Communication Services,28.777999,10.0,27.58
8,BIG,1.098273e+09,27.660000,Consumer Discretionary,28.735928,10.0,22.79
9,NGHC,2.505675e+09,22.120001,Financials,28.111789,10.0,21.05


In [22]:
# Constructing buy orders

amount = 837643

buy_orders['Account ID'] = 1777463
buy_orders['Asset Identifier'] = buy_orders['TICKER']
buy_orders['Asset Identifier Type'] = 'TICKER'
buy_orders['Transaction'] = 'Buy'
buy_orders['Trade Date'] = '2/5/2020'
buy_orders['Amount'] = (amount / 45) / buy_orders['Price']
buy_orders['Amount'] = buy_orders['Amount'].apply(np.floor)
buy_orders['Amount Type'] = 'SHARES'
buy_orders['Initial Purchase'] = 'Y'
buy_orders2 = buy_orders[['Account ID','Asset Identifier','Asset Identifier Type','Transaction','Trade Date','Amount','Amount Type','Initial Purchase','Price']]
buy_orders2

,Account ID,Asset Identifier,Asset Identifier Type,Transaction,Trade Date,Amount,Amount Type,Initial Purchase,Price
0,1777463,TSE,TICKER,Buy,2/5/2020,805.0,SHARES,Y,23.11
1,1777463,CNO,TICKER,Buy,2/5/2020,1075.0,SHARES,Y,17.30
2,1777463,MD,TICKER,Buy,2/5/2020,1076.0,SHARES,Y,17.29
3,1777463,SIG,TICKER,Buy,2/5/2020,818.0,SHARES,Y,22.74
4,1777463,LNC,TICKER,Buy,2/5/2020,363.0,SHARES,Y,51.18
5,1777463,AMCX,TICKER,Buy,2/5/2020,611.0,SHARES,Y,30.45
6,1777463,SC,TICKER,Buy,2/5/2020,716.0,SHARES,Y,25.97
7,1777463,MDP,TICKER,Buy,2/5/2020,674.0,SHARES,Y,27.58
8,1777463,BIG,TICKER,Buy,2/5/2020,816.0,SHARES,Y,22.79
9,1777463,NGHC,TICKER,Buy,2/5/2020,884.0,SHARES,Y,21.05


In [ ]:
buy_orders2.to_csv('trades_buyOrder_Portion_with_prices.csv',index=False)